In [ ]:
from google.colab import  drive
drive.mount('drive')

Mounted at drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
missing_values = ['?', '--', ' ', 'NA', 'N/A', '-']
data = pd.read_csv('drive/MyDrive/Colab Notebooks/academic/cardio_train.csv',delimiter = ';', na_values = missing_values)

In [ ]:
data

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


In [ ]:
data.drop(columns=['id'], inplace=True)

In [ ]:
duplicate_sum = data.duplicated().sum()
duplicate_sum

24

In [ ]:
data.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
out_filter = ((data["ap_hi"]>250) | (data["ap_lo"]>200))
data = data[~out_filter]

In [ ]:
out_filter1 = ((data["ap_hi"] < 0) | (data["ap_lo"] < 0))
data = data[~out_filter1]

In [ ]:
data

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [ ]:
Y = data['cardio']
X = data.drop(['cardio'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_tune, X_valid, y_tune, y_valid = train_test_split(X_train, y_train, random_state=42, test_size=0.2)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


**KNN Algorithm**

In [ ]:
leaf_size = list(range(5,10))
n_neighbors = list(range(5,10))
p=[1,2]
#Convert to dictionary
knn_params= dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
knn_grid = GridSearchCV(KNeighborsClassifier(metric='minkowski'), knn_params, cv=10)
knn_grid.fit(X_tune, y_tune)
knn_tuned_accuracy = knn_grid.score(X_valid,y_valid)
knn_tuned_params = knn_grid.best_params_

print(f"KNN tuned validation accuracy: {knn_tuned_accuracy:.3f}")
print(f"Selected hyperparameters for KNN: {knn_tuned_params}")

KNN tuned validation accuracy: 0.711
Selected hyperparameters for KNN: {'leaf_size': 5, 'n_neighbors': 9, 'p': 2}


In [ ]:
knn_final_acc=KNeighborsClassifier(metric='minkowski',n_neighbors=knn_tuned_params['n_neighbors'],leaf_size=knn_tuned_params['leaf_size'],p=2,weights='uniform')
knn_final_acc.fit(X_train,y_train)

final_acc=knn_final_acc.score(X_test,y_test)
print(f"KNN final test set accuracy: {final_acc:.4f}")

KNN final test set accuracy: 0.7142


**DecisionTreeClassifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_params={'max_depth':[1,2,3,4,5,7,10],"max_leaf_nodes":[4,5,6]}
dtc = GridSearchCV(DecisionTreeClassifier(random_state=42),decision_params,cv=5)
dtc.fit(X_tune, y_tune)
dtc_tuned_accuracy=dtc.score(X_valid,y_valid);
dtc_tuned_params=dtc.best_params_

print(f"Decision tree classifier tuned validation accuracy: {dtc_tuned_accuracy:.3f}")
print(f"Selected hyperparameters for Decision tree classifier: {dtc_tuned_params}")


Decision tree classifier tuned validation accuracy: 0.726
Selected hyperparameters for Decision tree classifier: {'max_depth': 3, 'max_leaf_nodes': 6}


In [ ]:
dtc_final_acc=DecisionTreeClassifier(random_state=42,max_depth=dtc_tuned_params['max_depth'],max_leaf_nodes=dtc_tuned_params['max_leaf_nodes'])
dtc_final_acc.fit(X_train,y_train)

final_acc=dtc_final_acc.score(X_test,y_test)

print(f"Decision tree classifier final accuracy:{final_acc:.3f}")

Decision tree classifier final accuracy:0.728


**SVM**

In [ ]:
from sklearn.svm import SVC
svm_params = {'C': 10.0**np.arange(1, 6), 'gamma': 10.0**np.arange(1, 6)}
svm_grid = GridSearchCV(SVC(kernel='rbf'), svm_params, cv=5)
svm_grid.fit(X_tune, y_tune)
svm_tuned_accuracy=svm_grid.score(X_valid,y_valid)
svm_tuned_params= svm_grid.best_params_

print(f"SVM tuned validation accuracy: {svm_tuned_accuracy:.3f}")
print(f"Selected hyperparameters For SVM: {svm_tuned_params}")


In [ ]:
svm_final_acc=SVC(C=svm_tuned_params['C'],gamma=svm_tuned_params['gamma'],random_state=42)
svm_final_acc.fit(X_train,y_train)

final_acc=svm_final_acc.score(X_test,y_test)
print(f"SVM final test set accuracy: {final_acc:.3f}")

**Naive bayes Gaussian Algorithm**

In [ ]:
from sklearn.naive_bayes import GaussianNB

Gnb_params={'var_smoothing': np.logspace(0,-9, num=100)}
Gnb_grid=GridSearchCV(GaussianNB(),Gnb_params,verbose=1,cv=5,n_jobs=-1)
Gnb_grid.fit(X_tune,y_tune)
Gnb_tuned_accuracy=Gnb_grid.score(X_valid,y_valid)
Gnb_tuned_params=Gnb_grid.best_params_

print(f"Gnb tuned validation accuracy:{Gnb_tuned_accuracy:.3f}")
print(f"Selected hyperparameters for GaussianNB: {Gnb_tuned_params}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Gnb tuned validation accuracy:0.703
Selected hyperparameters for GaussianNB: {'var_smoothing': 0.0015199110829529332}


In [ ]:
Gnb_final_acc=GaussianNB(var_smoothing=Gnb_tuned_params['var_smoothing'])
Gnb_final_acc.fit(X_train,y_train)
final_acc=Gnb_final_acc.score(X_test,y_test)

print(f"GaussianNB final test set accuracy:{final_acc:.3f}")

GaussianNB final test set accuracy:0.703


**RandomForestClassifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:

rfc_params = [{'max_depth': list(range(10, 15)), 'max_features': list(range(0,14))}]

rfc = GridSearchCV(RandomForestClassifier(), rfc_params, cv = 5, scoring='accuracy')

rfc.fit(X_tune,y_tune)
rfc_tuned_accuracy=rfc.score(X_valid,y_valid)
rfc_tuned_params=rfc.best_params_

print(f"rfc tuned validation accuracy:{rfc_tuned_accuracy:.3f}")
print(f"Selected hyperparameters for Random forest classsifier: {rfc_tuned_params}")

In [ ]:
rfc_final_acc=RandomForestClassifier(max_depth=rfc_tuned_params['max_depth'],max_features=rfc_tuned_params['max_features'])
rfc_final_acc.fit(X_train,y_train)
final_acc=rfc_final_acc.score(X_test,y_test)


In [ ]:
import pickle
pickle.dump(dtc_final_acc,open('CKD_model.pkl', 'wb'))

In [ ]:
loaded_model = pickle.load(open('CKD_model.pkl', 'rb'))

In [ ]:
loaded_model

DecisionTreeClassifier(max_depth=3, max_leaf_nodes=6, random_state=42)